In [1]:
import json
import pandas as pd
import jieba
from jieba import posseg
import numpy as np
from loader import *
import pickle
import torch
from torch import nn
import random
import sys
# from modules import embedding, encoder, model_sbj
from sklearn.model_selection import KFold

In [4]:
[1,2,[1,4]] == [1,2,[1,2]]

False

In [2]:
file_path = '../data/dev_data.json'

In [3]:
a = gen_test_data_p(file_path)

In [5]:
index = 0
print(a[0][index])
print(a[1][index])
print(a[2][index])

['查', '尔', '斯', '·', '阿', '兰', '基', '斯', '（', 'charles', ' ', 'ar', 'á', 'nguiz', '）', '，', '1', '9', '8', '9', '年', '4', '月', '1', '7', '日', '出', '生', '于', '智', '利', '圣', '地', '亚', '哥', '，', '智', '利', '职', '业', '足', '球', '运', '动', '员', '，', '司', '职', '中', '场', '，', '效', '力', '于', '德', '国', '足', '球', '甲', '级', '联', '赛', '勒', '沃', '库', '森', '足', '球', '俱', '乐', '部']
[(0, 7), (0, 7), (0, 7), (0, 7), (0, 7)]
[(29, 34), (54, 55), (16, 25), (29, 30), (36, 37)]


In [ ]:
text, sbj_bounds, obj_bounds, ps = batch

            # 裁剪
            text_mask = torch.ne(text, 0)
            max_len = text_mask.sum(dim=1).max().item()
            text_mask = text_mask[:, :max_len]
            text = text[:, :max_len]

            # embedding
            text_emb = self.embedding(text).transpose(0, 1)

            # encoder
            text_vec = self.encoder(text_emb, text_mask)

            # 构造p训练数据
            text_vec = text_vec.transpose(0, 1)
            batch_size = text.size(0)
            max_samples = sbj_bounds.size(1)
            x = []
            y = []
            for i in range(batch_size):
                vec = text_vec[i]
                sbj_bound = sbj_bounds[i]
                obj_bound = obj_bounds[i]
                p = ps[i]

                for j in range(max_samples):
                    if sbj_bound[j].sum() < 0:
                        break
                    vec_sbj_tmp = vec.index_select(dim=0, index=sbj_bound[j]).mean(dim=0)
                    vec_obj_tmp = vec.index_select(dim=0, index=obj_bound[j]).mean(dim=0)
                    vec_tmp = torch.cat([vec_sbj_tmp, vec_obj_tmp]).reshape(1, -1)
                    x.append(vec_tmp)
                    y.append(p[j].reshape(1, -1))
            x = torch.cat(x, dim=0)  # (*, h*4)
            y = torch.cat(y, dim=0)  # (*, 49)

            # p
            x = self.p_fc(x)
            loss_p = F.binary_cross_entropy_with_logits(x, y.float())